In [163]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from sklearn.metrics import classification_report, accuracy_score

In [164]:
train_data_dir = "Kidney_stone_detection-main/Kidney_stone_detection-main/Dataset/Train/"

In [174]:
img_size = (256, 256)  # adjust based on your image size
batch_size = 10 ## changed batch size from 30 to 10 to 5

In [166]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [167]:
# Load the training data using flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
#     target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'  # adjust if you have more than two classes
)

Found 1453 images belonging to 2 classes.


In [168]:
len(train_generator[0][0][0][0])

256

In [169]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_size[0], img_size[1], 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # adjust the number of units for more classes

In [170]:
custom_optimizer = Adam(learning_rate=0.0001)

In [171]:
# Compile the model
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [172]:
# Train the model
model.fit(train_generator, epochs=10)

Epoch 1/10
146/146 [==============================] - 112s 758ms/step - loss: 0.6781 - accuracy: 0.5871
Epoch 2/10
146/146 [==============================] - 86s 588ms/step - loss: 0.6502 - accuracy: 0.6187
Epoch 3/10
146/146 [==============================] - 86s 587ms/step - loss: 0.6115 - accuracy: 0.6710
Epoch 4/10
146/146 [==============================] - 85s 581ms/step - loss: 0.5746 - accuracy: 0.7061
Epoch 5/10
146/146 [==============================] - 85s 583ms/step - loss: 0.5380 - accuracy: 0.7433
Epoch 6/10
146/146 [==============================] - 88s 603ms/step - loss: 0.4957 - accuracy: 0.7632
Epoch 7/10
146/146 [==============================] - 90s 615ms/step - loss: 0.4615 - accuracy: 0.7887
Epoch 8/10
146/146 [==============================] - 85s 581ms/step - loss: 0.4312 - accuracy: 0.8052
Epoch 9/10
146/146 [==============================] - 85s 583ms/step - loss: 0.4086 - accuracy: 0.8142
Epoch 10/10
146/146 [==============================] - 84s 576ms/step - 

In [173]:
# Set the path to your test data
test_data_dir = "Kidney_stone_detection-main/Kidney_stone_detection-main/Dataset/Test/"

# Use ImageDataGenerator for normalization (no data augmentation for testing)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the test data using flow_from_directory
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'  # adjust if you have more than two classes
)

# Evaluate the model on the test data
evaluation = model.evaluate(test_generator)

# Display the evaluation metrics
print("Test Accuracy:", evaluation[1])
print("Test Loss:", evaluation[0])


Found 346 images belonging to 2 classes.
35/35 [==============================] - 10s 289ms/step - loss: 0.4684 - accuracy: 0.7659
Test Accuracy: 0.7658959627151489
Test Loss: 0.46836522221565247


In [154]:
y_pred = model.predict(test_generator)
y_pred_classes = (y_pred > 0.5).astype(int)  # Assuming binary classification, adjust threshold if needed

# Get true labels for the test data
y_true = test_generator.classes

12/12 [==============================] - 8s 665ms/step


In [162]:
report = classification_report(y_true, y_pred_classes, target_names=["Normal", "Kidney_stone"])  # Adjust class names
print(report)

              precision    recall  f1-score   support

      Normal       0.48      0.45      0.47       165
Kidney_stone       0.52      0.55      0.54       181

    accuracy                           0.50       346
   macro avg       0.50      0.50      0.50       346
weighted avg       0.50      0.50      0.50       346

